In [ ]:
#return 순서: 마지막 레이어(t-ensemble용), 학습 완료한 model, tr, ts predictions, tr accuracy, sensitivity, specificity, ts accuracy, sensitivity, specificity], 

def train_early_stopping_model(DataFrame x_val, pandas.core.series.Series y_val, 
                               pandas.core.frame.DataFrame test_x_val, pandas.core.series.Series test_y_val,
                               list layers, int drop_out, int input_drop_out, int learn_rate, int count, int batch_size):

    tr_loss_best = 100
    cnt=0
    best_model=[]
    features = shape(x_val)[1]
    input_layer = Input(shape=(features,))
    m_dp = Dropout(input_drop_out)(input_layer)

    for i in layers:
        m_layer = Dense(i,activation='relu')(m_dp)
        m_dp = Dropout(drop_out)(m_layer)

    final_layer = m_dp
    output_layer = Dense(1, activation="sigmoid")(final_layer)
    model_final = Model(inputs=input_layer,outputs=output_layer)
    model_final.compile(optimizer=tf.train.AdamOptimizer(learning_rate=learn_rate), 
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    while 1:
        model_final.fit(x_val, y_val, batch_size=batch_size, nb_epoch=1)
        tr_loss=model_final.evaluate(x_val,y_val)[0]
        if tr_loss < tr_loss_best:
            tr_loss_best = tr_loss
            count=0
            best_model = model_final
        else if cnt>count:
            model_final = best_model
            break
        else: cnt=cnt+1

    tr_accuracy=model_final.evaluate(x_val,y_val)[1]
    ts_accuracy= model_final.evaluate(test_x_val,test_y_val)[1]

    tr_predictions = model_final.predict(x_val)
    tr_labeled_predictions = np.where(tr_predictions > 0.5, 1, 0).flatten()
    tr_sensitivity, tr_specificity = check_correct(tr_labeled_predictions, y_val)

    ts_predictions = model_final.predict(test_x_val)
    ts_labeled_predictions = np.where(ts_predictions > 0.5, 1, 0).flatten()
    ts_sensitivity, ts_specificity = check_correct(ts_labeled_predictions, test_y_val)
    
    return final_layer, model_final, tr_predictions, ts_predictions, tr_accuracy, tr_sensitivity, tr_specificity, ts_accuracy, ts_sensitivity, ts_specificity

